In [1]:
import kaggle as kg
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as s
import os

In [2]:
os.environ["USERNAME"] = "aimlrl"
os.environ["KEY"] = "f9fcb3d12c2c4845724a50f0cdeab5ea"

In [4]:
kg.api.authenticate()

In [ ]:
kg.api.dataset_download_files(dataset="uciml/breast-cancer-wisconsin-data",
                              path="./dataset",unzip=True)

In [2]:
data = pd.read_csv("dataset/data.csv")

In [ ]:
data

In [ ]:
data.isna().sum()

In [ ]:
data.columns

In [6]:
data.drop(labels=[data.columns[0],data.columns[-1]],axis=1,inplace=True)

In [ ]:
data

In [ ]:
data[data.columns[0]].replace(to_replace=['B','M'],value=[0,1],
                              inplace=True)

In [ ]:
data

In [ ]:
data_corr = data.corr()
upper_traingle_mask = np.triu(np.ones_like(data_corr,dtype=bool))
sns.heatmap(data=data_corr,center=0,annot=True,square=True,xticklabels=True,yticklabels=True,mask=upper_traingle_mask)
plt.show()

In [11]:
selected_features = list(data_corr[(data_corr[data.columns[0]] < -0.5) | (data_corr[data.columns[0]] > 0.5)].index)

In [ ]:
selected_features

In [13]:
refined_data = data[selected_features]

In [ ]:
refined_data

In [23]:
X_transpose = np.array(refined_data.iloc[:,1:])
X_transpose = (X_transpose - np.mean(X_transpose,axis=0))/np.std(X_transpose,axis=0)
y = np.array(refined_data.iloc[:,0]).reshape(refined_data.shape[0],1)

In [ ]:
pd.DataFrame(X_transpose)

In [24]:
def sigmoid(theta0,theta):

    return 1/(1 + np.exp(-(theta0 + np.matmul(X_transpose,theta))))

In [25]:
def bce(theta0,theta):

    p = sigmoid(theta0,theta)
    first_term = np.matmul(np.transpose(y),np.log(p))
    second_term = np.matmul(np.transpose(1-y),np.log(1-p))

    return -(1/p.shape[0])*(first_term + second_term)

In [26]:
def del_bce_by_del_theta(theta0,theta):

    p = sigmoid(theta0,theta)
    error_transpose = np.transpose(p - y)

    del_by_del_theta0 = np.mean(error_transpose)
    del_by_del_theta = (1/y.shape[0])*np.transpose(np.matmul(error_transpose,X_transpose))

    return [del_by_del_theta0,del_by_del_theta]

In [ ]:
epsilon = 10**(-4)
epoch_counter = 0
theta0_initial = 0
theta_initial = np.zeros((X_transpose.shape[1],1))
tol = 10**(-5)

while True:

    initial_gradients = del_bce_by_del_theta(theta0_initial,theta_initial)

    theta0_final = theta0_initial - (epsilon * initial_gradients[0])
    theta_final = theta_initial - (epsilon * initial_gradients[1])

    bce_initial_value = bce(theta0_initial,theta_initial)
    bce_final_value = bce(theta0_final,theta_final)

    if abs(bce_initial_value - bce_final_value) < tol:
        break

    epoch_counter += 1

    theta0_initial = theta0_final
    theta_initial = theta_final

    print("Epoch # {}, BCE Value = {}".format(epoch_counter,bce_initial_value[0,0]))